In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, datetime
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve, auc, confusion_matrix
from scipy.optimize import minimize

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras.utils import plot_model
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, classification_report, roc_curve
import utils

In [2]:
#person = 'Davis'
person = 'JOURNEY HOUSE TRAVEL INC'
cat_vars = ['Merchant Category Code (MCC)']
numeric_vars = ['Amount']
categories = ['Cardholder Last Name', 'Cardholder First Initial'] + numeric_vars + cat_vars

In [3]:
df = utils.get_df()

In [4]:
transations = utils.get_person_true_transations_df(df, person, categories, cat_vars, numeric_vars)
df_transations = transations[0][0]
labels = transations[1][0]

In [5]:
x_train, x_test, y_train, y_test = train_test_split(df_transations, labels, test_size=.2, random_state=42)

# Normalize the testing and training data using the MinMaxScaler from the scikit learn package
scaler = MinMaxScaler()

# Make sure to only fit the scaler on the training data
x_train = scaler.fit_transform(df_transations)
x_test = scaler.transform(x_test)
# convert the data to FP32
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

input_dim = x_train.shape[1]

In [6]:
# model hyperparameters
latent_dim = 10
max_epochs = 15
learning_rate = 0.001


opt = optimizers.Adam(learning_rate=learning_rate)
activator = tf.keras.layers.LeakyReLU(alpha=0.01)
#activator = "relu"

In [7]:
def create_model():
    input_data = Input(shape=(input_dim,), name='encoder_input')
    
    dimensions = [200, 100, 30]

    encoder = Dense(dimensions[0],activation=activator, name='encoder_1')(input_data)
    encoder = Dropout(.1)(encoder)
    lay = 1
    for num in dimensions[1:]:
        lay += 1
        encoder = Dense(num,activation=tf.keras.layers.LeakyReLU(alpha=0.01), name='encoder_'+str(lay))(encoder)
        encoder = Dropout(.1)(encoder)
        
    latent_encoding = Dense(latent_dim, activation='linear', name='latent_encoding')(encoder)

    decoder = Dense(dimensions[-1] , activation=tf.keras.layers.LeakyReLU(alpha=0.01), name='decoder_1')(latent_encoding)
    decoder = Dropout(.1)(decoder)
    dimensions.pop()

    lay = 1
    for num in dimensions[::-1]:
        lay += 1
        decoder = Dense(num, activation=tf.keras.layers.LeakyReLU(alpha=0.01), name='decoder_'+ str(lay))(decoder)
        decoder = Dropout(.1)(decoder)
    
    reconstructed_data = Dense(input_dim, activation='linear', name='reconstructed_data')(decoder)
    
    autoencoder_model = Model(input_data, reconstructed_data)
    encoder_model = Model(input_data, latent_encoding)
    
    autoencoder_model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])
    encoder_model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])
    return autoencoder_model, encoder_model

In [8]:
autoencoder_model, encoder_model = create_model()

2022-11-28 21:28:30.518692: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 21:28:30.519608: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
autoencoder_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 435)]             0         
                                                                 
 encoder_1 (Dense)           (None, 200)               87200     
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 encoder_2 (Dense)           (None, 100)               20100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 encoder_3 (Dense)           (None, 30)                3030      
                                                                 
 dropout_2 (Dropout)         (None, 30)                0     

In [ ]:
batch_size = len(x_train)//10
train_history = autoencoder_model.fit(x_train, x_train,
        shuffle=True,
        epochs=max_epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test)
        )

In [ ]:
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.legend(['loss on train data', 'loss on validation data'])

Autoencoder efficience

In [ ]:
x_test_recon = autoencoder_model.predict(x_test)
reconstruction_scores = np.mean((x_test - x_test_recon)**2, axis=1)
plt.xlabel('Reconstruction Score')
df_a = pd.DataFrame()
df_a['recon_score'] = reconstruction_scores
minim, maxim = df_a['recon_score'].min(), df_a['recon_score'].max()
df_a['recon_score'].plot.hist(bins=200, range=[minim, maxim])

In [ ]:
transations_val = utils.get_person_balanced_df(df, person, categories, cat_vars, numeric_vars)
df_transations_val = transations_val[0][0]
labels_val = transations_val[1][0]
scaler = MinMaxScaler()
x_test_val = scaler.fit_transform(df_transations_val)

x_test_recon_val = autoencoder_model.predict(x_test_val)
reconstruction_scores_val = np.mean((x_test_val - x_test_recon_val)**2, axis=1)

In [ ]:
tpr, fpr, thresholds = roc_curve(labels_val, reconstruction_scores_val)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='lime', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
optimal_threshold_idx = np.argmax(tpr - fpr)
threshold = thresholds[optimal_threshold_idx]
print(threshold)

Validate Model

In [ ]:
transations_val_2 = utils.get_person_total_df(df, person, categories, cat_vars, numeric_vars)
df_transations_val_2 = transations_val_2[0][0]
labels_val_2 = transations_val_2[1][0]
scaler = MinMaxScaler()
x_test_val_2 = scaler.fit_transform(df_transations_val_2)

x_test_recon = autoencoder_model.predict(x_test_val_2)
reconstruction_scores_2 = np.mean((x_test_val_2 - x_test_recon)**2, axis=1)
anomaly_data = pd.DataFrame({'recon_score':reconstruction_scores_2})
pred_labels_2 = (reconstruction_scores_2 < threshold).astype(int)

In [ ]:
accuracy_score(pred_labels_2, labels_val_2)

In [ ]:
plt.xlabel('Reconstruction Score')
df_a = pd.DataFrame()
df_a['recon_score'] = reconstruction_scores_2
minim, maxim = df_a['recon_score'].min(), df_a['recon_score'].max()
df_a['recon_score'].plot.hist(bins=200, range=[minim, maxim])

In [ ]:
print ('Confusion Matrix: ')
results = confusion_matrix(labels_val_2, pred_labels_2) 
utils.plot_confusion_matrix(results, ['Anomaly','Normal'])

## For all persons

In [24]:
def calc_accuracy(name: str):
    #Train
    transations = utils.get_person_true_transations_df(df, name, categories, cat_vars, numeric_vars)
    if len(transations[0]) == 0:
        return -1
    df_transations = transations[0][0]
    labels = transations[1][0]
    x_train, x_test, y_train, y_test = train_test_split(df_transations, labels, test_size=.2, random_state=42)
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(df_transations)
    x_test = scaler.transform(x_test)
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    autoencoder_model, encoder_model = create_model()
    batch_size = len(x_train)//10
    train_history = autoencoder_model.fit(x_train, x_train,
        shuffle=True,
        epochs=max_epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test)
        #callbacks=[tensorboard_callback]
        )
    #Find Threshold
    transations_val = utils.get_person_balanced_df(df, person, categories, cat_vars, numeric_vars)
    df_transations_val = transations_val[0][0]
    labels_val = transations_val[1][0]
    scaler = MinMaxScaler()
    x_test_val = scaler.fit_transform(df_transations_val)
    x_test_recon_val = autoencoder_model.predict(x_test_val)
    reconstruction_scores_val = np.mean((x_test_val - x_test_recon_val)**2, axis=1)
    tpr, fpr, thresholds = roc_curve(labels_val, reconstruction_scores_val)
    optimal_threshold_idx = np.argmax(tpr - fpr)
    threshold = thresholds[optimal_threshold_idx]
    # Validate
    transations_val_2 = utils.get_person_total_df(df, person, categories, cat_vars, numeric_vars)
    df_transations_val_2 = transations_val_2[0][0]
    labels_val_2 = transations_val_2[1][0]
    scaler = MinMaxScaler()
    x_test_val_2 = scaler.fit_transform(df_transations_val_2)

    x_test_recon = autoencoder_model.predict(x_test_val_2)
    reconstruction_scores_2 = np.mean((x_test_val_2 - x_test_recon)**2, axis=1)
    anomaly_data = pd.DataFrame({'recon_score':reconstruction_scores_2})
    pred_labels_2 = (reconstruction_scores_2 < threshold).astype(int)

    return accuracy_score(labels_val_2, pred_labels_2)

In [25]:
persons = list(df['Cardholder Last Name'].unique())
quantities = df['Cardholder Last Name'].value_counts()
values = []
names = []

for person in persons:
    if quantities[person] > 1000:
        accuracy = calc_accuracy(person)
        if accuracy != -1:
            values.append(accuracy) 
            names.append(quantities[person])

Epoch 1/15
11/11 [==============================] - 1s 61ms/step - loss: 0.0023 - accuracy: 0.2110 - val_loss: 0.0022 - val_accuracy: 0.2374
Epoch 2/15
11/11 [==============================] - 0s 35ms/step - loss: 0.0021 - accuracy: 0.2402 - val_loss: 0.0018 - val_accuracy: 0.2374
Epoch 3/15
11/11 [==============================] - 0s 32ms/step - loss: 0.0019 - accuracy: 0.2849 - val_loss: 0.0018 - val_accuracy: 0.3151
Epoch 4/15
11/11 [==============================] - 0s 27ms/step - loss: 0.0018 - accuracy: 0.3352 - val_loss: 0.0016 - val_accuracy: 0.3562
Epoch 5/15
11/11 [==============================] - 0s 39ms/step - loss: 0.0017 - accuracy: 0.3881 - val_loss: 0.0015 - val_accuracy: 0.4566
Epoch 6/15
11/11 [==============================] - 0s 37ms/step - loss: 0.0016 - accuracy: 0.4484 - val_loss: 0.0015 - val_accuracy: 0.5023
Epoch 7/15
11/11 [==============================] - 0s 26ms/step - loss: 0.0015 - accuracy: 0.4904 - val_loss: 0.0014 - val_accuracy: 0.5023
Epoch 8/15
11

In [ ]:
plt.figure()
plt.plot(names, values, ".")
plt.title("Accuracy vs Operations Quantities")
plt.xlabel("Operations Quantities")
plt.ylabel("Accuracy")

In [ ]:
values

In [ ]:
names